# 🌀 Hurricane Ian WRF Analysis
This Jupyter notebook demonstrates how to:
1. Load WRF model data.
2. Find the hurricane center using different methods.
3. Plot the storm and overlay the center.
4. Compute vortex-centered coordinates.
5. Decompose winds into radial and tangential components.

Each step is modular and can be run independently.

In [ ]:
import netCDF4 as nc
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime
from pyproj import Proj, Transformer
from wrf import getvar, interplevel
from pathlib import Path

## To install this environment, use the following commands:
# conda create -n wrfenv python=3.11 wrf-python xarray netCDF4 cartopy matplotlib pyproj -c conda-forge
# conda activate wrfenv

## 1️⃣ Load WRF Dataset

In [ ]:
data_path = Path('data/wrfout_d03_2022-09-26_Ian2022_UNCPL.nc')
ds = nc.Dataset(data_path)
lat = ds.variables['XLAT'][0, :, :]
lon = ds.variables['XLONG'][0, :, :]
print(f"Loaded WRF file with shape: {lat.shape}")

## 2️⃣ Find Hurricane Center (Surface Pressure Minimum)

In [ ]:
psfc = ds.variables['PSFC'][-1, :, :] / 100  # Pa → hPa
imin = np.unravel_index(np.argmin(psfc), psfc.shape)
center_lat_psfc, center_lon_psfc = float(lat[imin]), float(lon[imin])
print(f"Center (min PSFC): {center_lat_psfc:.2f}°, {center_lon_psfc:.2f}°")

## 3️⃣ Find Hurricane Center (850-hPa Vorticity Maximum)

In [ ]:
wrf_ds = xr.open_dataset(data_path)
p = getvar(wrf_ds, 'pressure')
avo = getvar(wrf_ds, 'avo', units='1e-5 s-1')
avo850 = interplevel(avo, p, 850)
lat_wrf, lon_wrf = getvar(wrf_ds, 'lat'), getvar(wrf_ds, 'lon')
imax = np.unravel_index(np.argmax(avo850.values), avo850.shape)
center_lat_vort, center_lon_vort = float(lat_wrf.values[imax]), float(lon_wrf.values[imax])
print(f"Center (max 850-hPa vorticity): {center_lat_vort:.2f}°, {center_lon_vort:.2f}°")

## 4️⃣ Find Hurricane Center (10-m Wind Speed Minimum)

In [ ]:
u10 = ds.variables['U10'][-1, :, :]
v10 = ds.variables['V10'][-1, :, :]
wind_speed = np.sqrt(u10**2 + v10**2)
imin_ws = np.unravel_index(np.argmin(wind_speed), wind_speed.shape)
center_lat_ws, center_lon_ws = float(lat[imin_ws]), float(lon[imin_ws])
print(f"Center (min 10-m wind): {center_lat_ws:.2f}°, {center_lon_ws:.2f}°")

## 5️⃣ Create Vortex-Centered Azimuthal Equidistant Projection

In [ ]:
center_lat, center_lon = center_lat_psfc, center_lon_psfc  # use PSFC center
aeqd = Proj(proj='aeqd', lat_0=center_lat, lon_0=center_lon, datum='WGS84')
transformer = Transformer.from_proj(ccrs.PlateCarree().proj4_init, aeqd, always_xy=True)
x, y = transformer.transform(lon, lat)
r = np.sqrt(x**2 + y**2) / 1000  # km
theta = np.degrees(np.arctan2(y, x))
print('Computed storm-centered r and θ grids.')

## 6️⃣ Compute Radial and Tangential Wind Components

In [ ]:
u = u10
v = v10
# Convert geographic wind to storm-relative components
radial_wind =  u * np.cos(np.radians(theta)) + v * np.sin(np.radians(theta))
tangential_wind = -u * np.sin(np.radians(theta)) + v * np.cos(np.radians(theta))
print('Radial and tangential components computed.')

## 7️⃣ Plot Wind Speed, Vectors, and Center

In [ ]:
fig, ax = plt.subplots(figsize=(12,10), subplot_kw={'projection': ccrs.PlateCarree()})
ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()])
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.contourf(lon, lat, wind_speed, levels=np.linspace(0, 60, 21), cmap='bone')
ax.quiver(lon[::20,::20], lat[::20,::20], u10[::20,::20], v10[::20,::20], scale=1500)
ax.plot(center_lon, center_lat, 'r*', markersize=20, label='Center')
plt.legend()
plt.title('Hurricane Ian - 10m Wind Field and Center')
plt.show()